# Facility Location Problem

Our problem involves transporting supply from facilities to consumers, for example shipping products from factories to retail locations. Let there be $M$ customers and $N$ facilities. Each customer $i = 1, \ldots, M$ needs to be shipped $f_i$ units, and each facility $j$ has the capacity to produce $b_j$ units. The cost of transporting one unit of supply from facility $j$ to customer $i$ is given by $d_{ij}$, the distance between them. Our goal is to transport the goods from the facilities to the customers at minimum cost.

The twist is that none of these facilities have been built yet, and we can only build $K$ facilities in total. Goods cannot be shipped from a facility that is not build. This means we also need to decide which facilities to build in order to deliver the goods at minimal cost.

## Problem Formulation

\begin{align*}
\min~~ & \sum_{i=1}^M \sum_{j=1}^N d_{ij} x_{ij} \\
\text{s.t.} ~~ & x_{ij} \leq b_j y_j, && \forall \, i =1,\dots, M, \ j =1,\dots, N, \\
& \sum_{j=1}^N x_{ij} = f_i, && \forall \, i =1,\dots,M,\\
& \sum_{i=1}^M x_{ij} \leq b_j, && \forall \, j =1,\dots,N,\\
& \sum_{j=1}^N y_j \leq K, \\
& x_{ij} \geq 0, && \forall \, i =1,\dots, M, \ j = 1,\dots, N, \\
& y_j \in \{ 0, 1\}, && \forall \, j =1,\dots, N.
\end{align*}

### Decision variables

Let $x_{ij}$ be the supply from facility $j$ to customer $i$ (we have $x_{ij} \ge 0$).

Let $y_j$ be a binary variable indicating if facility $j$ is built.

### Constraints

We need each customer $i$ to receive $f_i$ units of supply:

$\sum_{j=1}^N x_{ij} = f_i, ~~~\forall \, i =1,\dots,M$

We need each facility $j$ to produce at most $b_j$ units of supply:

$\sum_{i=1}^M x_{ij} \leq b_j, ~~~\forall \, i =1,\dots,M$

We restrict ourselves to $K$ facilities being built:

$\sum_{j=1}^N y_j \leq K$

We cannot send supply from facility $j$ unless it is built:

$ x_{ij} \leq b_j y_j,~~~ \forall \, i =1,\dots, M, \ j =1,\dots, N$

This is an example of a big-M constraint: if $y_j = 1$, $x_{ij}$ can be as large as it wants (there is no way it can be higher than $b_j$ though due to the supply constraint above), and if $y_j = 0$, then $x_{ij}$ is forced to be zero.

### Objective

We want to minimize the total cost:

$\min \sum_{i=1}^M \sum_{j=1}^N d_{ij} x_{ij}$


## Part I: Solving the problem in JuMP

Suppose we live in a 1D world, where customers and facilities are located on a line, and $d_{ij}$ is just the distance between customer $i$ and facility $j$ on the line. Let's solve the problem for the following data:

In [33]:
customer_locs = [3; 7; 9; 10; 12; 15; 18; 20]
facility_locs = [1; 5; 10; 12; 24]
f = [1; 1; 6; 8; 6; 10; 6; 10]
b = [22; 18; 15; 21; 23]
K = 3
M = length(customer_locs)
N = length(facility_locs)

5

In [34]:
rand(15:25, 5)

5-element Array{Int64,1}:
 25
 19
 17
 21
 23

Calculate distances:

In [35]:
d = [abs(customer_locs[i] - facility_locs[j]) for i = 1:M, j = 1:N]

8×5 Array{Int64,2}:
  2   2   7  9  21
  6   2   3  5  17
  8   4   1  3  15
  9   5   0  2  14
 11   7   2  0  12
 14  10   5  3   9
 17  13   8  6   6
 19  15  10  8   4

Let's make a model in JuMP!

In [36]:
using JuMP, Gurobi
model = Model(solver=GurobiSolver(OutputFlag=0))

# Add variables
@variable(model, x[i = 1:M, j = 1:N] >= 0)
@variable(model, y[j = 1:N], Bin)

# Add constraints
@constraint(model, mustbuild[i = 1:M, j = 1:N], x[i, j] <= b[j] * y[j])
@constraint(model, demand[i = 1:M], sum(x[i, j] for j = 1:N) == f[i])
@constraint(model, sum(y[j] for j = 1:N) <= K)

# Add objective
@objective(model, Min, sum(x[i, j] * d[i, j] for i = 1:M, j = 1:N))

2 x[1,1] + 2 x[1,2] + 7 x[1,3] + 9 x[1,4] + 21 x[1,5] + 6 x[2,1] + 2 x[2,2] + 3 x[2,3] + 5 x[2,4] + 17 x[2,5] + 8 x[3,1] + 4 x[3,2] + x[3,3] + 3 x[3,4] + 15 x[3,5] + 9 x[4,1] + 5 x[4,2] + 2 x[4,4] + 14 x[4,5] + 11 x[5,1] + 7 x[5,2] + 2 x[5,3] + 12 x[5,5] + 14 x[6,1] + 10 x[6,2] + 5 x[6,3] + 3 x[6,4] + 9 x[6,5] + 17 x[7,1] + 13 x[7,2] + 8 x[7,3] + 6 x[7,4] + 6 x[7,5] + 19 x[8,1] + 15 x[8,2] + 10 x[8,3] + 8 x[8,4] + 4 x[8,5]

In [37]:
# Solve the model
status = solve(model)
status

:Optimal

In [38]:
# Check the optimal variable values
getvalue(x)

8×5 Array{Float64,2}:
 0.0  0.0  1.0   0.0   0.0
 0.0  0.0  1.0   0.0   0.0
 0.0  0.0  6.0   0.0   0.0
 0.0  0.0  8.0   0.0   0.0
 0.0  0.0  0.0   6.0   0.0
 0.0  0.0  0.0  10.0   0.0
 0.0  0.0  0.0   6.0   0.0
 0.0  0.0  0.0   0.0  10.0

In [39]:
getvalue(y)

5-element Array{Float64,1}:
 -0.0
  0.0
  1.0
  1.0
  1.0

In [40]:
# Check the optimal objective value
getobjectivevalue(model)

122.0

## Part II - Finding next best solutions

In MIPs, it is sometimes useful to know not only the best solution, but also the second best, third best and so on.

Now add code to solve the model three more times. Each time, add a constraint that cuts off the current facility decision, re-solve the model and output the objective value.

**Hint 1**: Think about how you might add a constraint to exclude a single solution?

**Hint 2**: Suppose our current solution is $\bar y$ with
$$
  J_1 = \{ j ~|~ \bar y_j = 1 \} ~~~ J_2 = \{ j ~|~ \bar y_j = 0 \}
$$

The following constraint will cut off the current solution:
$$
  \sum_{j \in J_1} y_j + \sum_{j \in J_0} (1 - y_j) \le N - 1
$$

In [41]:
for rep = 1:3
    y_bar = getvalue(y)
    J_1 = find(y_bar .== 1)
    J_0 = find(y_bar .== 0)
    
    @constraint(model, sum(y[j] for j in J_1) + sum(1 - y[j] for j in J_0) <= N - 1)
    
    solve(model)
    println(getobjectivevalue(model), " ", getvalue(y))
end

144.0 [-0.0, 1.0, 0.0, 1.0, 1.0]
147.0 [1.0, -0.0, 0.0, 1.0, 1.0]
148.00000000000003 [-0.0, 1.0, 1.0, 0.0, 1.0]
